In [ ]:
import requests
from bs4 import BeautifulSoup
import csv

baseurl = "https://www.venbafragrance.com/"

# Headers for request
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36'
}

productlinks = []
for x in range(1, 8):
    r = requests.get(f'https://www.venbafragrance.com/collections/all-bottles?page={x}', headers=headers)
    soup = BeautifulSoup(r.content, "lxml")
    productlist = soup.find_all('div', class_='product-grid-item')

    for item in productlist:
        for link in item.find_all('a', href=True):
            productlinks.append(baseurl.rstrip('/') + link['href'])

# Prepare to write to CSV
with open('perfumes.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    # Write the header row
    writer.writerow(['Brand', 'Rating', 'Reviews', 'Price', 'Image Link'])

    for link in productlinks:
        r = requests.get(link, headers=headers)
        soup = BeautifulSoup(r.content, 'lxml')

        # Extracting product details
        brand = soup.find('h1', class_='product__title').text.strip() if soup.find('h1', class_='product__title') else "Brand Not Found"
        rating = float(soup.find('span', class_='jdgm-prev-badge__stars')['data-score']) if soup.find('span', class_='jdgm-prev-badge__stars') else 0.0
        reviews = soup.find('span', class_='jdgm-prev-badge__text').text.strip() if soup.find('span', class_='jdgm-prev-badge__text') else "No Reviews Found"
        price = soup.find('span', class_='product__price').text.strip() if soup.find('span', class_='product__price') else "Price Not Available"

        # Extracting the image URL
        img_tag = soup.find('img', {'data-product-image': True})  # Assuming there is a specific attribute
        image_link = img_tag['src'] if img_tag else "Image link not found"
        if image_link.startswith('//'):
            image_link = 'https:' + image_link  # Handling protocol-relative URLs

        # Writing row to CSV
        writer.writerow([brand, rating, reviews, price, image_link])

# print("Data has been successfully exported to 'perfumes.csv'.")


Data has been successfully exported to 'perfumes.csv'.
